<a href="https://colab.research.google.com/github/LEChaney/ML-RayCast-Experiments/blob/master/supervised_learning_4_localization_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/LEChaney/ML-RayCast-Experiments
%cd ML-RayCast-Experiments
%pwd

Cloning into 'ML-RayCast-Experiments'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (112/112), done.
remote: Total 329 (delta 59), reused 24 (delta 7), pack-reused 206
Receiving objects: 100% (329/329), 46.01 MiB | 11.49 MiB/s, done.
Resolving deltas: 100% (167/167), done.
/content/ML-RayCast-Experiments


'/content/ML-RayCast-Experiments'

In [0]:
from PIL import Image
def rotate(image_path):
    """
    Rotate the given photo the amount of given degreesk, show it and save it
    @param image_path: The path to the image to edit
    @param degrees_to_rotate: The number of degrees to rotate the image
    @param saved_location: Path to save the cropped image
    """
    degrees_to_rotate = 180
    image_obj = Image.open(image_path)
    rotated_image = image_obj.rotate(degrees_to_rotate)
    rotated_image.save("000.png")
    im = cv2.imread("000.png")
    #plt.imshow(im)
    #plt.show()
    return im
    


In [0]:
import os
os.environ['SDL_VIDEODRIVER']='dummy'
file_directory = os.getcwd()
os.chdir("/content/ML-RayCast-Experiments/Keras-FlappyBird-master/Keras-FlappyBird-master")
print(os.getcwd())

/content/ML-RayCast-Experiments/Keras-FlappyBird-master/Keras-FlappyBird-master


In [0]:
import sys
#sys.path.pop(1)
sys.path.insert(1,"/content/ML-RayCast-Experiments/Keras-FlappyBird-master/Keras-FlappyBird-master/game")
print(sys.path)

['', '/content/ML-RayCast-Experiments/Keras-FlappyBird-master/Keras-FlappyBird-master/game', '/env/python', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython']


In [0]:
%pip install pygame

     |████████████████████████████████| 11.4MB 2.7MB/s 


In [0]:
import wrapped_flappy_bird as game
import numpy as np
import flappy_bird_utils
import cv2
import sys
import random
import pygame
import pygame.surfarray as surfarray
from pygame.locals import *
from itertools import cycle
import matplotlib.pyplot as plt
import PIL.Image
#from cStringIO import StringIO
import IPython.display

import skimage as skimage
from skimage import transform, color, exposure
from skimage.transform import rotate
from skimage.viewer import ImageViewer

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


/usr/local/lib/python3.6/dist-packages/skimage/viewer/__init__.py:6: UserWarning: Viewer requires Qt
  warn('Viewer requires Qt')


In [0]:
GAME = 'bird' # the name of the game being played for log files
CONFIG = 'nothreshold'
ACTIONS = 2 # number of valid actions
GAMMA = 0.99 # decay rate of past observations
OBSERVATION = 3200. # timesteps to observe before training
EXPLORE = 3000000. # frames over which to anneal epsilon
FINAL_EPSILON = 0.0001 # final value of epsilon
INITIAL_EPSILON = 0.1 # starting value of epsilon
REPLAY_MEMORY = 50000 # number of previous transitions to remember
BATCH = 32 # size of minibatch
FRAME_PER_ACTION = 1
LEARNING_RATE = 1e-4

FPS = 30
SCREENWIDTH  = 288
SCREENHEIGHT = 512

pygame.init()
pygame.display.init()
pygame.display.set_mode((1, 1))
FPSCLOCK = pygame.time.Clock()
SCREEN = pygame.Surface((SCREENWIDTH, SCREENHEIGHT)).convert_alpha()
pygame.display.set_caption('Flappy Bird')

IMAGES, SOUNDS, HITMASKS = flappy_bird_utils.load()
PIPEGAPSIZE = 100 # gap between upper and lower part of pipe
BASEY = SCREENHEIGHT * 0.79

PLAYER_WIDTH = IMAGES['player'][0].get_width()
PLAYER_HEIGHT = IMAGES['player'][0].get_height()
PIPE_WIDTH = IMAGES['pipe'][0].get_width()
PIPE_HEIGHT = IMAGES['pipe'][0].get_height()
BACKGROUND_WIDTH = IMAGES['background'].get_width()

PLAYER_INDEX_GEN = cycle([0, 1, 2, 1])

img_rows , img_cols = 80, 80
#Convert image into Black and white
img_channels = 4 #We stack 4 frames

game_state = game.GameState()
do_nothing = np.zeros(ACTIONS)
do_nothing[0] = 1
game_state.frame_step(do_nothing)

(array([[[0, 0, 0]]], dtype=uint8), 0.1, False)

In [0]:
def get_image_from_coords(data_list):
  
    CaseNo = int(float(data_list[0]))
    game_state.playerx = int(float(data_list[1]))# + SCREENWIDTH /2)
    game_state.playery = int(float(data_list[2]))# + SCREENWIDTH /2)
    game_state.playerIndex = int(data_list[3])        

    game_state.upperPipes = []
    game_state.lowerPipes = []

    for i in range(4,len(data_list),4):

        newPipe = game.getRandomPipe()
        game_state.upperPipes.append(newPipe[0])
        game_state.upperPipes[int((i-4)/4)]['x'] = int(float(data_list[i]))# + SCREENWIDTH /2)
        game_state.upperPipes[int((i-4)/4)]['y'] = int(float(data_list[i+1]))# + SCREENWIDTH /2)

        game_state.lowerPipes.append(newPipe[1])
        game_state.lowerPipes[int((i-4)/4)]['x'] = int(float(data_list[i+2]))# + SCREENWIDTH /2)
        game_state.lowerPipes[int((i-4)/4)]['y'] = int(float(data_list[i+3]))# + SCREENWIDTH /2)

    # draw sprites
    SCREEN.blit(IMAGES['background'], (0,0))

    for uPipe, lPipe in zip(game_state.upperPipes, game_state.lowerPipes):
        SCREEN.blit(IMAGES['pipe'][0], (uPipe['x'], uPipe['y']))
        SCREEN.blit(IMAGES['pipe'][1], (lPipe['x'], lPipe['y']))

    SCREEN.blit(IMAGES['base'], (game_state.basex, BASEY))
    # print score so player overlaps the score
    # showScore(self.score)
    SCREEN.blit(IMAGES['player'][game_state.playerIndex],
                (game_state.playerx, game_state.playery))

    image_data = pygame.surfarray.array3d(SCREEN)
    #IPython.display.display(PIL.Image.fromarray(image_data))

    image_data = skimage.color.rgb2gray(image_data)
    image_data = skimage.transform.resize(image_data,(80,80))
    image_data = skimage.exposure.rescale_intensity(image_data, out_range=(0,255))

    image_data = image_data / 255.0

    image_data = image_data.reshape(1, image_data.shape[0], image_data.shape[1], 1) #1x80x80x1

    label = [game_state.playerx, game_state.playery]
    
    return image_data, label

def gen_batch(file, bs, D):
    
    images=[]
    labels=[]
        
    state = []
    state_labels = []

    while len(images) < bs:  
        
        current = file.readline()

        #if reach end of the file start again
        if current == "":
            
            file.seek(0)

            current = file.readline()

            data_list = current.split(',')
            data_list.pop()

            image_data, state_label = get_image_from_coords(data_list)
            state = np.stack((image_data, image_data, image_data, image_data), axis=2)
            state = state.reshape(1, state.shape[0], state.shape[1], state.shape[2])  #1*80*80*4
            state_label = np.array(state_label)
            state_labels = np.stack((label, label, label, label), axis=2).flatten()

        #state = state.reshape(state.shape[0],)
                # store the transition in D
            D.append((state, state_labels))   
            
#             if current[0] == "-":
#                 continue
        else:   
            
            data_list = current.split(',')
            data_list.pop()

            image_data, state_label = get_image_from_coords(data_list)

    #             state.append(image_data)
    #             state_labels.append(label)

            state = np.append(image_data, state[:, :, :, :3], axis=3)
            state_labels = np.append(state_label, state_labels[:6]).flatten()
            #state = np.array(state)
            #state = state.reshape(state.shape[0],)
        
        # store the transition in D
        D.append((state, state_labels))
        if len(D) > REPLAY_MEMORY:
            D.popleft()

        gen_state, gen_label = random.sample(D,1)
        images.append(gen_state)
        labels.append(gen_label)
            
    # if the data augmentation object is not None, apply it
    if aug is not None:
      (images, labels) = next(aug.flow(np.array(images), labels, batch_size=bs))
    
        #running = False
    #print(type(image_data))
    print("image_data" ,image_data)
    print("image_data.shape: ", image_data.shape)
    IPython.display.display(PIL.Image.fromarray(image_data))
    #plt.show()
    
    return images, labels

SyntaxError: ignored

In [0]:

'''import numpy as np
import sys
import random
import pygame
import flappy_bird_utils
import pygame.surfarray as surfarray
from pygame.locals import *
from itertools import cycle'''

def csv_image_generator(filepath, bs, mode="train", aug=None):

    file = open(filepath,"r")
    
    maxCount = 100
    count = maxCount
    lineNum = 0
    
    # store the previous observations in replay memory
    D = deque()
    
    # initialize state
    current = file.readline()

    data_list = current.split(',')
    data_list.pop()

    image_data, state_label = get_image_from_coords(data_list)
    state = np.stack((image_data, image_data, image_data, image_data), axis=2)
    state = state.reshape(1, state.shape[0], state.shape[1], state.shape[2])  #1*80*80*4
    state_label = np.array(state_label)
    state_labels = np.stack((label, label, label, label), axis=2).flatten()
    
    #state = state.reshape(state.shape[0],)
        # store the transition in D
    D.append((state, state_labels)) 
    
    while True:
        
        count -= 1

#         if count > 1:
#             continue
#         if count == 0:
#             break;
        
        images, labels = get_batch(file, bs, D)
        
        # yield the batch to the calling function
        yield (np.array(images), np.array(labels))    
    

In [0]:
file= "data.txt" #file path or a file in the current folder
csv_image_generator(file, 8, mode="train", aug=None)

<generator object csv_image_generator at 0x7f6805691990>

In [0]:
# USAGE
# python train.py

# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
import sys
from __future__ import print_function

from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, adam
from keras.initializers import normal, identity
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from tensorflow.keras import Model
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from tensorflow.keras.layers import Conv2D, Reshape
from tensorflow.keras.utils import Sequence
from tensorflow.keras.backend import epsilon

import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report

from pyimagesearch.minivggnet import MiniVGGNet

import matplotlib.pyplot as plt

import random
import numpy as np

from coord import CoordinateChannel2D

from sklearn.model_selection import train_test_split

img_rows , img_cols = 80, 80
#Convert image into Black and white
img_channels = 4 #We stack 4 frames

usingSimpleCNN = 1
usingSimpleCoordConv = 0
usingMobileNetV2Localizer = 0

def buildSimpleCNN():
    print("Now we build the model")
    model = Sequential()
    model.add(Convolution2D(32, 8, 8, subsample=(4,4), border_mode='same', input_shape=(img_rows,img_cols,img_channels)))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, 4, 4, subsample=(2,2), border_mode='same'))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, 3, 3, subsample=(1,1), border_mode='same'))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(8))
   
    adam = Adam(lr=1e-6)
    model.compile(loss='mse',optimizer=adam)
    print("We finish building the model")
    return model

def create_COORDCONV_Simple_CNN():
    print("Now we build the model")
    ip = (img_rows,img_cols,img_channels)
    x = CoordinateChannel2D()(ip)
    x = Convolution2D(32, 8, 8, subsample=(4,4),init=lambda shape, name: normal(shape, scale=0.01, name=name), border_mode='same')
    x = (Activation('relu')
    x = CoordinateChannel2D()(x)
    x = Convolution2D(64, 4, 4, subsample=(2,2),init=lambda shape, name: normal(shape, scale=0.01, name=name), border_mode='same')
    x = Activation('relu')
    x = CoordinateChannel2D()(x)
    x = Convolution2D(64, 3, 3, subsample=(1,1),init=lambda shape, name: normal(shape, scale=0.01, name=name), border_mode='same')
    x = Activation('relu')
    x = Flatten()
    x = Dense(512, init=lambda shape, name: normal(shape, scale=0.01, name=name))
    x = Activation('relu')
    x = Dense(8,init=lambda shape, name: normal(shape, scale=0.01, name=name))
    model = keras.Model(inputs=inputs, outputs=x)
    adam = Adam(lr=1e-6)
    model.compile(loss='mse',optimizer=adam)
    return model

"""
# prior to first conv
ip = Input(shape=(64, 64, 2))
x = CoordinateChannel2D()(ip)
x = Conv2D(...)(x)  # This defines the `CoordConv` from the paper.
...
x = CoordinateChannel2D(use_radius=True)(x)
x = Conv2D(...)(x)  # This adds the 3rd channel for the radius.
"""
def create_MobileNetV2_model(trainable=False):
    model = MobileNetV2(input_shape=(img_channels,img_rows,img_cols), include_top=False)

    # to freeze layers
    for layer in model.layers:
        layer.trainable = trainable

    x = model.layers[-1].output
    x = Conv2D(4, kernel_size=3, name="coords")(x)
    x = Reshape((8,))(x)

    model = Model(inputs=model.input, outputs=x)
    adam = Adam(lr=1e-6)
    model.compile(loss='mse', optimizer=adam)

    return model

def get_train_test_split():
    data = np.genfromtxt('Keras-FlappyBird-master/Keras-FlappyBird-master/data.csv', delimiter=',')
#    data = np.random.shuffle(data)
    traintestsplit = np.split(data, int(data.shape[0]/5))
    train = traintestsplit[0]
    test = traintestsplit[1]
    return train, test

# initialize the paths to our training and testing CSV files
#TRAIN_CSV = "flowers17_training.csv"
#TEST_CSV = "flowers17_testing.csv"

TRAIN, TEST = get_train_test_split()

# initialize the number of epochs to train for and batch size
NUM_EPOCHS = 75
BS = 32

# initialize the total number of training and testing image
NUM_TRAIN_IMAGES = 0
NUM_TEST_IMAGES = 0

# open the training CSV file, then initialize the unique set of class
# labels in the dataset along with the testing labels
#f = open(TRAIN_CSV, "r")
labels = set()
testLabels = []

'''
# loop over all rows of the CSV file
for row in TRAIN:
	# extract the class label, update the labels list, and increment
	# the total number of training images
	label = line.strip().split(",")[0]
	labels.add(label)
	NUM_TRAIN_IMAGES += 1

# close the training CSV file and open the testing CSV file
f.close()
f = open(TEST_CSV, "r")

# loop over the lines in the testing file
for row in TEST:
	# extract the class label, update the test labels list, and
	# increment the total number of testing images
	label = line.strip().split(",")[0]
	testLabels.append(label)
	NUM_TEST_IMAGES += 1

# close the testing CSV file
f.close()
'''

trainLabels = TRAIN[:,1:3]
trainData = TRAIN[:,:]
np.savetxt('trainData.csv', trainData, delimiter=',')
NUM_TRAIN_IMAGES = trainData.shape[0]

testLabels = TEST[:,1:3]
testData = TEST[:,:]
np.savetxt('testData.csv', testData, delimiter=',')
NUM_TEST_IMAGES = testData.shape[0]

# create the label binarizer for one-hot encoding labels, then encode
# the testing labels
# lb = LabelBinarizer()
# lb.fit(list(trainLabels))
# testLabels = lb.transform(testLabels)

# construct the training image generator for data augmentation
aug = ImageDataGenerator(width_shift_range=0.2, height_shift_range=0.2, fill_mode="nearest")

# initialize both the training and testing image generators
trainGen = csv_image_generator('trainData.csv', BS, #lb,
	mode="train", aug=aug)
testGen = csv_image_generator('testData.csv', BS, #lb,
	mode="train", aug=None)

# initialize our Keras model and compile it
if usingSimpleCNN:
    model = buildSimpleCNN()
elif usingSimpleCoordConv:
    model = create_COORDCONV_Simple_CNN()    
elif usingMobileNetV2Localizer:
    model = create_MobileNetV2_model()

#    model = MiniVGGNet.build(64, 64, 3, len(lb.classes_))
#   opt = SGD(lr=1e-2, momentum=0.9, decay=1e-2 / NUM_EPOCHS)
#   model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

# train the network
print("[INFO] training w/ generator...")
H = model.fit_generator(
	trainGen,
	steps_per_epoch=NUM_TRAIN_IMAGES // BS,
	validation_data=testGen,
	validation_steps=NUM_TEST_IMAGES // BS,
	epochs=NUM_EPOCHS)

# re-initialize our testing data generator, this time for evaluating
testGen = csv_image_generator(TEST_CSV, BS, #lb,
	mode="eval", aug=None)

# make predictions on the testing images, finding the index of the
# label with the corresponding largest predicted probability
predIdxs = model.predict_generator(testGen,
	steps=(NUM_TEST_IMAGES // BS) + 1)
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print("[INFO] evaluating network...")
print(classification_report(testLabels.argmax(axis=1), predIdxs,
	target_names=lb.classes_))

# plot the training loss and accuracy
N = NUM_EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")

SyntaxError: ignored